In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta

# Load Dataset
df = pd.read_csv("final_dataset.csv")
df.rename(columns={'date': 'Date', 'demand': 'Demand'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Sort dataset by date
df = df.sort_index()

# Select Target Variable
target_col = "Demand"
data = df[[target_col]].dropna()

# Normalize Data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 30  # Use past 30 days for training
X, y = create_sequences(data_scaled, seq_length)

# Split Data into Training & Testing
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile & Train Model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Forecast Function
def forecast_future(model, data, days):
    future_predictions = []
    last_sequence = data[-seq_length:].reshape(1, seq_length, 1)
    
    for _ in range(days):
        pred = model.predict(last_sequence)[0]
        future_predictions.append(pred)
        last_sequence = np.append(last_sequence[:, 1:, :], [[pred]], axis=1)
    
    return scaler.inverse_transform(future_predictions)

# Generate Forecasts
future_days_dict = {"1 Week": 7, "1 Month": 30, "1 Year": 365, "2 Year": 730,"3 Years": 1095}
future_forecasts = {}

for key, days in future_days_dict.items():
    future_forecasts[key] = forecast_future(model, data_scaled, days)

# Generate future dates
last_date = df.index[-1]
future_dates = {key: [last_date + timedelta(days=i+1) for i in range(days)] for key, days in future_days_dict.items()}

# Plot Forecasts
def plot_forecast(actual, future_dates, future_preds, title):
    plt.figure(figsize=(12, 5))
    plt.plot(actual.index, actual[target_col], label='Actual Demand', color='blue')
    plt.plot(future_dates, future_preds, label='Forecast', color='red', linestyle='dashed')
    plt.xlabel("Date")
    plt.ylabel("Electricity Demand")
    plt.title(title)
    plt.legend()
    plt.show()

# Plot Individual Forecasts
for key in future_days_dict.keys():
    plot_forecast(df, future_dates[key], future_forecasts[key], f"{key} Electricity Demand Forecast")

Epoch 1/50


E:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


104/104 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 0.0410 - val_loss: 0.0184
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0200 - val_loss: 0.0179
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0193 - val_loss: 0.0170
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0196 - val_loss: 0.0177
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0198 - val_loss: 0.0183
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0199 - val_loss: 0.0170
Epoch 7/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0198 - val_loss: 0.0172
Epoch 8/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0206 - val_loss: 0.0163
Epoch 9/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0188 - val_loss: 0.0154
Epoch 10/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0182 - val_loss: 0.0141
Epoch 11/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0150 - val_loss: 0.0117
Epoch 12/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/ste